## 第7章 数据清洗和准备

在本章中，我会讨论处理缺失数据、重复数据、字符串操作和其
它分析数据转换的⼯具。

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt



### 7.1 处理缺失数据


对于数值数据， pandas使⽤浮点值
NaN（Not a Number）表示缺失数据。我们称其为哨兵值，可以
⽅便的检测出来：

In [3]:
string_data = pd.Series(['asdfsdf', 'wtrafda', np.nan, 'asdfew'])
string_data
string_data.isnull()

0    False
1    False
2     True
3    False
dtype: bool

将缺失值表示为
NA，它表示不可⽤not available。在统计应⽤中， NA数据可能
是不存在的数据或者虽然存在，但是没有观察到（例如，数据采
集中发⽣了问题）。当进⾏数据清洗以进⾏分析时，最好直接对
缺失数据进⾏分析，以判断数据采集的问题或缺失数据可能导致
的偏差。

Python内置的None值在对象数组中也可以作为NA：

In [4]:
string_data[0] = None
string_data.isnull()

0     True
1    False
2     True
3    False
dtype: bool

### 滤除缺失数据

过滤掉缺失数据的办法有很多种。你可以通过pandas.isnull或布
尔索引的⼿⼯⽅法，但dropna可能会更实⽤⼀些。对于⼀个
Series， dropna返回⼀个仅含⾮空数据和索引值的Series：

In [5]:
from numpy import nan as NA
data = pd.Series([1, NA, 3.5, NA, 7])
data.dropna()

0    1.0
2    3.5
4    7.0
dtype: float64

这等价于

In [6]:
data[data.notnull()]

0    1.0
2    3.5
4    7.0
dtype: float64

⽽对于DataFrame对象，事情就有点复杂了。你可能希望丢弃全
NA或含有NA的⾏或列。 dropna默认丢弃任何含有缺失值的⾏：

In [7]:
data = pd.DataFrame([[1., 6.5, 3.], [1., NA, NA], [NA, NA, NA], [NA, 3.2, 12.]])
cleaned = data.dropna()
print(data)
print(cleaned)

     0    1     2
0  1.0  6.5   3.0
1  1.0  NaN   NaN
2  NaN  NaN   NaN
3  NaN  3.2  12.0
     0    1    2
0  1.0  6.5  3.0


传⼊how='all'将只丢弃全为NA的那些⾏

In [8]:
data.dropna(how='all')  # 全为NA的行被丢弃

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
3,NaN,3.2,12.0


In [9]:
data[4] = NA
print(data)
print()
data.dropna(axis=1, how='all')

     0    1     2   4
0  1.0  6.5   3.0 NaN
1  1.0  NaN   NaN NaN
2  NaN  NaN   NaN NaN
3  NaN  3.2  12.0 NaN



,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,3.2,12.0


另⼀个滤除DataFrame⾏的问题涉及时间序列数据。假设你只想
留下⼀部分观测数据，可以⽤thresh参数实现此⽬的

**其实thresh=n不是滤除n行，而且保留至少有n个非NaN数据的行**

In [10]:
np.random.seed(2018)

df = pd.DataFrame(np.random.randn(7, 3))
df.iloc[:4, 1] = NA
df.iloc[:2, 2] = NA
df.iloc[4:6, 0] = NA
print(df)
print()
print(df.dropna())
# print(df.dropna(thresh=2))
print()
print(df.dropna(thresh=1))
print()
print(df.dropna(thresh=2))
print()
print()
print(df.dropna(thresh=3))

          0         1         2
0 -0.276768       NaN       NaN
1 -1.279487       NaN       NaN
2 -0.142790       NaN -0.688065
3  0.433564       NaN -0.165131
4       NaN  0.546631  1.230655
5       NaN -1.210625 -0.306677
6 -1.057419  0.402057  0.289165

          0         1         2
6 -1.057419  0.402057  0.289165

          0         1         2
0 -0.276768       NaN       NaN
1 -1.279487       NaN       NaN
2 -0.142790       NaN -0.688065
3  0.433564       NaN -0.165131
4       NaN  0.546631  1.230655
5       NaN -1.210625 -0.306677
6 -1.057419  0.402057  0.289165

          0         1         2
2 -0.142790       NaN -0.688065
3  0.433564       NaN -0.165131
4       NaN  0.546631  1.230655
5       NaN -1.210625 -0.306677
6 -1.057419  0.402057  0.289165


          0         1         2
6 -1.057419  0.402057  0.289165


### 填充缺失数据
你可能不想滤除缺失数据（有可能会丢弃跟它有关的其他数
据），⽽是希望通过其他⽅式填补那些“空洞”。对于⼤多数情况
⽽⾔，

fillna⽅法是最主要的函数。通过⼀个常数调⽤fillna就会将
缺失值替换为那个常数值

In [11]:
df.fillna(0)

,0,1,2
0,-0.276768,0.000000,0.000000
1,-1.279487,0.000000,0.000000
2,-0.142790,0.000000,-0.688065
3,0.433564,0.000000,-0.165131
4,0.000000,0.546631,1.230655
5,0.000000,-1.210625,-0.306677
6,-1.057419,0.402057,0.289165


若是通过⼀个字典调⽤fillna，就可以实现对不同的列填充不同的
值：

In [12]:
df.fillna({1:0.5, 2:8.8})

,0,1,2
0,-0.276768,0.500000,8.800000
1,-1.279487,0.500000,8.800000
2,-0.142790,0.500000,-0.688065
3,0.433564,0.500000,-0.165131
4,NaN,0.546631,1.230655
5,NaN,-1.210625,-0.306677
6,-1.057419,0.402057,0.289165


fillna默认会返回新对象，但也可以对现有对象进⾏就地修改：

In [13]:
_ = df.fillna(0, inplace=True)
df

,0,1,2
0,-0.276768,0.000000,0.000000
1,-1.279487,0.000000,0.000000
2,-0.142790,0.000000,-0.688065
3,0.433564,0.000000,-0.165131
4,0.000000,0.546631,1.230655
5,0.000000,-1.210625,-0.306677
6,-1.057419,0.402057,0.289165


对reindexing有效的那些插值⽅法也可⽤于fillna：

In [14]:
df = pd.DataFrame(np.random.randn(6,3))
df.iloc[2:, 1] = NA
df.iloc[4:, 2] = NA
df

,0,1,2
0,1.282733,-1.065696,-1.706633
1,-0.172797,0.063710,0.370628
2,-1.604543,NaN,0.380370
3,-0.276501,NaN,0.421854
4,0.704780,NaN,NaN
5,0.138125,NaN,NaN


In [15]:
# method : {‘backfill’, ‘bfill’, ‘pad’, ‘ffill’, None}, default None
# pad/ffill：用前一个非缺失值去填充该缺失值
# backfill/bfill：用下一个非缺失值填充该缺失值
df.fillna(method='ffill')

,0,1,2
0,1.282733,-1.065696,-1.706633
1,-0.172797,0.063710,0.370628
2,-1.604543,0.063710,0.380370
3,-0.276501,0.063710,0.421854
4,0.704780,0.063710,0.421854
5,0.138125,0.063710,0.421854


In [16]:
df.fillna(method='ffill', limit=2)

,0,1,2
0,1.282733,-1.065696,-1.706633
1,-0.172797,0.063710,0.370628
2,-1.604543,0.063710,0.380370
3,-0.276501,0.063710,0.421854
4,0.704780,NaN,0.421854
5,0.138125,NaN,0.421854


In [17]:
# 还可以插入Series的平均值或中位数
data = pd.Series([1, NA, 3.5, 6.2, NA, 8])

data.fillna(data.mean())

0    1.000
1    4.675
2    3.500
3    6.200
4    4.675
5    8.000
dtype: float64

### 7.2数据转换

### 移除重复数据

In [18]:
data = pd.DataFrame({'k1': ['one', 'two'] * 3 + ['two'],
                     'k2': [1, 1, 2, 3, 3, 4, 4]})
data

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4
6,two,4


DataFrame的duplicated⽅法返回⼀个布尔型Series，表示各⾏
是否是重复⾏（前⾯出现过的⾏）

In [19]:
data.duplicated()

0    False
1    False
2    False
3    False
4    False
5    False
6     True
dtype: bool

还有⼀个与此相关的drop_duplicates⽅法，它会返回⼀个
DataFrame，重复的数组会标为False

In [20]:
data.drop_duplicates()

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4


这两个⽅法默认会判断全部列，你也可以指定部分列进⾏重复项
判断。假设我们还有⼀列值，且只希望根据k1列过滤重复项：

In [21]:
data['v1'] = range(7)
data

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
5,two,4,5
6,two,4,6


In [22]:
data.drop_duplicates(['k1'])

,k1,k2,v1
0,one,1,0
1,two,1,1


duplicated和drop_duplicates默认保留的是第⼀个出现的值组
合。传⼊keep='last'则保留最后⼀个

In [23]:
data.drop_duplicates(['k1', 'k2'], keep='last')

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
6,two,4,6


### 利用函数映射进行数据转换

中的值来实现转换⼯作。我们来看看下⾯这组有关⾁类的数据：

In [24]:
data = pd.DataFrame({'food': ['bacon', 'pulled pork', 'bacon',
                              'Pastrami', 'corned beef', 'Bacon',
                              'pastrami', 'honey ham', 'nova lox'],
                     'ounces': [4, 3, 12, 6, 7.5, 8, 3, 5, 6]})
data

,food,ounces
0,bacon,4.0
1,pulled pork,3.0
2,bacon,12.0
3,Pastrami,6.0
4,corned beef,7.5
5,Bacon,8.0
6,pastrami,3.0
7,honey ham,5.0
8,nova lox,6.0


假设你想要添加⼀列表示该⾁类⻝物来源的动物类型。我们先编
写⼀个不同⾁类到动物的映射：

In [25]:

meat_to_animal = {
  'bacon': 'pig',
  'pulled pork': 'pig',
  'pastrami': 'cow',
  'corned beef': 'cow',
  'honey ham': 'pig',
  'nova lox': 'salmon'
}

Series的map⽅法可以接受⼀个函数或含有映射关系的字典型对
象，但是这⾥有⼀个⼩问题，即有些⾁类的⾸字⺟⼤写了，⽽另
⼀些则没有。因此，我们还需要使⽤Series的str.lower⽅法，将
各个值转换为⼩写：

In [26]:
lowercased = data['food'].str.lower()
lowercased

0          bacon
1    pulled pork
2          bacon
3       pastrami
4    corned beef
5          bacon
6       pastrami
7      honey ham
8       nova lox
Name: food, dtype: object

In [27]:
data['animal'] = lowercased.map(meat_to_animal)
data

,food,ounces,animal
0,bacon,4.0,pig
1,pulled pork,3.0,pig
2,bacon,12.0,pig
3,Pastrami,6.0,cow
4,corned beef,7.5,cow
5,Bacon,8.0,pig
6,pastrami,3.0,cow
7,honey ham,5.0,pig
8,nova lox,6.0,salmon


我们也可以传⼊⼀个能够完成全部这些⼯作的函数：

In [28]:
data['food'].map(lambda x: meat_to_animal[x.lower()])
data

,food,ounces,animal
0,bacon,4.0,pig
1,pulled pork,3.0,pig
2,bacon,12.0,pig
3,Pastrami,6.0,cow
4,corned beef,7.5,cow
5,Bacon,8.0,pig
6,pastrami,3.0,cow
7,honey ham,5.0,pig
8,nova lox,6.0,salmon


### 替换值

In [29]:
data = pd.Series([1., -999., 2., -999., -1000., 3.])
data

0       1.0
1    -999.0
2       2.0
3    -999.0
4   -1000.0
5       3.0
dtype: float64

-999这个值可能是⼀个表示缺失数据的标记值。要将其替换为
pandas能够理解的NA值，

我们可以利⽤replace来产⽣⼀个新的
Series（除⾮传⼊inplace=True）

In [30]:
data.replace(-999, np.nan)

0       1.0
1       NaN
2       2.0
3       NaN
4   -1000.0
5       3.0
dtype: float64

如果你希望⼀次性替换多个值，可以传⼊⼀个由待替换值组成的
列表以及⼀个替换值：

In [31]:
data.replace([-999, -1000], np.nan)

0    1.0
1    NaN
2    2.0
3    NaN
4    NaN
5    3.0
dtype: float64

要让每个值有不同的替换值，可以传递⼀个替换列表

In [32]:
data.replace([-999, -1000], [np.nan, 0])

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

传⼊的参数也可以是字典：


笔记： data.replace⽅法与data.str.replace不同，后者做的是
字符串的元素级替换。我们会在后⾯学习Series的字符串⽅
法。

In [33]:
data.replace({-999: np.nan, -1000: 0})

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

### 重命名轴索引

跟Series中的值⼀样，轴标签也可以通过函数或映射进⾏转换，
从⽽得到⼀个新的不同标签的对象。轴还可以被就地修改，⽽⽆
需新建⼀个数据结构。

In [34]:
data = pd.DataFrame(np.arange(12).reshape((3, 4)),
                    index=['Ohio', 'Colorado', 'New York'],
                    columns=['one', 'two', 'three', 'four'])
data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
New York,8,9,10,11


跟Series一样，轴索引也有一个map方法：

In [35]:
transform = lambda x: x[:4].upper()
data.index.map(transform)

Index(['OHIO', 'COLO', 'NEW '], dtype='object')

你可以将其赋值给index，这样就可以对DataFrame进⾏就地修
改

In [36]:
data.index = data.index.map(transform)
data

,one,two,three,four
OHIO,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


如果想要创建数据集的转换版（⽽不是修改原始数据），⽐较实
⽤的⽅法是rename：

In [37]:
data.rename(index=str.title, columns=str.upper)

,ONE,TWO,THREE,FOUR
Ohio,0,1,2,3
Colo,4,5,6,7
New,8,9,10,11


rename可以结合字典型对象实现对部分轴标签
的更新：

In [38]:
data.rename(index={'OHIO': 'INDIANA'}, columns={'three': 'peekaboo'})

,one,two,peekaboo,four
INDIANA,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


rename可以实现复制DataFrame并对其索引和列标签进⾏赋
值。如果希望就地修改某个数据集，传⼊inplace=True即可：

In [39]:
data.rename(index={'OHIO': 'INDIANA'}, inplace=True)
data

,one,two,three,four
INDIANA,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


### 离散化和面元划分
为了便于分析，连续数据常常被离散化或拆分为“⾯元”（bin）。
假设有⼀组⼈员数据，⽽你希望将它们划分为不同的年龄组：

In [40]:
ages = [20, 22, 25, 27, 21, 23, 37, 31, 61, 45, 41, 32]

接下来将这些数据划分为“18到25”、 “26到35”、 “35到60”以及“60
以上”⼏个⾯元。

要实现该功能，你需要使⽤pandas的cut函数：

In [41]:
bins = [18, 25, 35, 60, 100]
cats = pd.cut(ages, bins)
cats

[(18, 25], (18, 25], (18, 25], (25, 35], (18, 25], ..., (25, 35], (60, 100], (35, 60], (35, 60], (25, 35]]
Length: 12
Categories (4, interval[int64]): [(18, 25] < (25, 35] < (35, 60] < (60, 100]]

In [42]:
type(cats)

pandas.core.arrays.categorical.Categorical

pandas返回的是⼀个特殊的Categorical对象。结果展示了
pandas.cut划分的⾯元。你可以将其看做⼀组表示⾯元名称的字
符串。它的底层含有⼀个表示不同分类名称的类型数组，以及⼀
个codes属性中的年龄数据的标签：

In [43]:
cats.codes

array([0, 0, 0, 1, 0, 0, 2, 1, 3, 2, 2, 1], dtype=int8)

In [44]:
cats.categories

IntervalIndex([(18, 25], (25, 35], (35, 60], (60, 100]],
              closed='right',
              dtype='interval[int64]')

In [45]:
# pd.value_counts(cats)是pandas.cut结果的⾯元计数。
pd.value_counts(cats)

(18, 25]     5
(35, 60]     3
(25, 35]     3
(60, 100]    1
dtype: int64

跟“区间”的数学符号⼀样，圆括号表示开端，⽽⽅括号则表示闭
端（包括）。哪边是闭端可以通过right=False进⾏修改：

In [46]:
pd.cut(ages, [18, 26, 36, 61, 100], right=False)

[[18, 26), [18, 26), [18, 26), [26, 36), [18, 26), ..., [26, 36), [61, 100), [36, 61), [36, 61), [26, 36)]
Length: 12
Categories (4, interval[int64]): [[18, 26) < [26, 36) < [36, 61) < [61, 100)]

可以通过传递⼀个列表或数组到labels，设置⾃⼰的⾯元名
称：

In [51]:
group_names = ['Youth', 'YoungAdult', 'MiddleAged', 'Senior']
pd.cut(ages, bins, labels=group_names)

[Youth, Youth, Youth, YoungAdult, Youth, ..., YoungAdult, Senior, MiddleAged, MiddleAged, YoungAdult]
Length: 12
Categories (4, object): [Youth < YoungAdult < MiddleAged < Senior]

如果向cut传⼊的是⾯元的数量⽽不是确切的⾯元边界，则它会
根据数据的最⼩值和最⼤值计算等⻓⾯元。

In [54]:
# 下⾯这个例⼦中，我们将⼀些均匀分布的数据分成四组：
data = np.random.rand(20)

pd.cut(data, 4, precision=2)  # 选项precision=2， 限定小数只有2位

[(0.7, 0.93], (0.47, 0.7], (0.7, 0.93], (0.47, 0.7], (0.0021, 0.24], ..., (0.0021, 0.24], (0.7, 0.93], (0.7, 0.93], (0.0021, 0.24], (0.0021, 0.24]]
Length: 20
Categories (4, interval[float64]): [(0.0021, 0.24] < (0.24, 0.47] < (0.47, 0.7] < (0.7, 0.93]]

qcut是⼀个⾮常类似于cut的函数，它可以根据样本分位数对数据
进⾏⾯元划分。根据数据的分布情况， cut可能⽆法使各个⾯元
中含有相同数量的数据点。⽽qcut由于使⽤的是样本分位数，因
此可以得到⼤⼩基本相等的⾯元：

*四分位数（英语：Quartile）是统计学中分位数的一种，即把所有数值由小到大排列并分成四等份，处于三个分割点位置的数值就是四分位数。*

In [55]:
data = np.random.randn(1000)  # Normally distributed
cats = pd.qcut(data, 4)  # Cut into quartiles
cats

[(0.729, 3.327], (-4.317, -0.647], (-4.317, -0.647], (-0.647, 0.00594], (-4.317, -0.647], ..., (-4.317, -0.647], (-0.647, 0.00594], (0.00594, 0.729], (0.729, 3.327], (0.729, 3.327]]
Length: 1000
Categories (4, interval[float64]): [(-4.317, -0.647] < (-0.647, 0.00594] < (0.00594, 0.729] < (0.729, 3.327]]

In [57]:
pd.value_counts(cats)

(0.729, 3.327]       250
(0.00594, 0.729]     250
(-0.647, 0.00594]    250
(-4.317, -0.647]     250
dtype: int64

与cut类似，也可以传递自定义的分位数（0到1之间的数值，包含端点）

In [59]:
pd.qcut(data, [0, 0.1, 0.5, 0.9, 1.])   # integer or array of quantiles

[(0.00594, 1.355], (-1.24, 0.00594], (-4.317, -1.24], (-1.24, 0.00594], (-1.24, 0.00594], ..., (-1.24, 0.00594], (-1.24, 0.00594], (0.00594, 1.355], (0.00594, 1.355], (0.00594, 1.355]]
Length: 1000
Categories (4, interval[float64]): [(-4.317, -1.24] < (-1.24, 0.00594] < (0.00594, 1.355] < (1.355, 3.327]]

### 检测和过滤异常值
过滤或变换异常值（outlier）在很⼤程度上就是运⽤数组运算

In [62]:
data = pd.DataFrame(np.random.randn(1000, 4))
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.001346,-0.005334,0.001282,0.004753
std,1.015783,1.008176,1.002819,0.980442
min,-3.314143,-3.495468,-3.911487,-2.817035
25%,-0.680172,-0.672127,-0.662892,-0.649029
50%,-0.007743,-0.021555,-0.012393,-0.009470
75%,0.662183,0.664017,0.687181,0.701375
max,3.679034,3.316661,2.758841,3.380704


假设你想要找出某列中绝对值大小超过3的值

In [63]:
col = data[2]
col[np.abs(col) > 3]

553   -3.911487
784   -3.007361
Name: 2, dtype: float64

要选出全部含有“超过3或-3的值”的行，你可一在布尔型DataFrame中使用any方法

In [81]:
data[(np.abs(data) > 3).any(1)]  # axis : {0 or ‘index’, 1 or ‘columns’, None}, default 0
# pandas.DataFrame.any() Return whether any element is True, potentially over an axis.

,0,1,2,3
142,3.014600,1.009981,0.316872,0.205827
293,3.679034,-0.020092,0.113900,-0.765671
345,-0.851273,0.360798,0.439062,3.380704
415,-2.452382,-3.495468,-0.207652,0.694295
553,-0.491008,-0.792561,-3.911487,-0.808454
784,1.729500,-0.277903,-3.007361,0.158534
826,0.409614,3.316661,0.376361,-0.296493
982,-3.314143,0.732287,0.064483,-0.452210


根据这些条件，就可以对值进⾏设置。下⾯的代码可以将值限制
在区间－ 3到3以内：

In [83]:
data[np.abs(data) > 3] = np.sign(data) * 3
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.000967,-0.005155,0.002201,0.004372
std,1.012523,1.005582,0.999646,0.979202
min,-3.000000,-3.000000,-3.000000,-2.817035
25%,-0.680172,-0.672127,-0.662892,-0.649029
50%,-0.007743,-0.021555,-0.012393,-0.009470
75%,0.662183,0.664017,0.687181,0.701375
max,3.000000,3.000000,2.758841,3.000000


根据数据的值是正还是负， np.sign(data)可以⽣成1和-1：

In [84]:
np.sign(data).head()

,0,1,2,3
0,-1.0,-1.0,-1.0,-1.0
1,-1.0,1.0,-1.0,-1.0
2,-1.0,-1.0,-1.0,-1.0
3,1.0,1.0,-1.0,-1.0
4,-1.0,1.0,-1.0,1.0


### 排列和随机采样
利⽤numpy.random.permutation函数可以轻松实现对Series或
DataFrame的列的排列⼯作（permuting，随机重排序）。

通过需要排列的轴的⻓度调⽤permutation，可产⽣⼀个表示新顺序
的整数数组：

In [85]:
df = pd.DataFrame(np.arange(5 * 4).reshape((5,4)))
sampler = np.random.permutation(5)
sampler

array([3, 0, 1, 2, 4])

可以在基于iloc的索引操作或take函数中使用该数组：

In [86]:
df

,0,1,2,3
0,0,1,2,3
1,4,5,6,7
2,8,9,10,11
3,12,13,14,15
4,16,17,18,19


In [87]:
df.take(sampler)

,0,1,2,3
3,12,13,14,15
0,0,1,2,3
1,4,5,6,7
2,8,9,10,11
4,16,17,18,19


如果不想⽤替换的⽅式选取随机⼦集，可以在Series和
DataFrame上使⽤sample⽅法

In [96]:
df.sample(n=3)

,0,1,2,3
2,8,9,10,11
4,16,17,18,19
3,12,13,14,15


要通过替换的⽅式产⽣样本（允许重复选择），可以传递
replace=True到sample：

In [90]:
choices = pd.Series([5, 7, -1, 6, 4])
draws = choices.sample(n=10, replace=True)
draws

0    5
4    4
0    5
0    5
1    7
1    7
4    4
0    5
0    5
0    5
dtype: int64

### 计算指标/哑变量
另⼀种常⽤于统计建模或机器学习的转换⽅式是：将分类变量（categorical variable）转换为“哑变量”或“指标矩阵”。如果DataFrame的某⼀列中含有k个不同的值，则可以派⽣出⼀个k列矩阵或DataFrame（其值全为1和0）。 

pandas有⼀个get_dummies函数可以实现该功能（其实⾃⼰动⼿做⼀个也不难）。使⽤之前的⼀个DataFrame例⼦

*虚拟变量 ( Dummy Variables) 又称虚设变量、名义变量或哑变量，用以反映质的属性的一个人工变量，是量化了的质变量，通常取值为0或1*

In [112]:
df = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'], 'data1': range(6)})
df

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,b,5


In [113]:
pd.get_dummies(df['key'])

,a,b,c
0,0,1,0
1,0,1,0
2,1,0,0
3,0,0,1
4,1,0,0
5,0,1,0


有时候，你可能想给指标DataFrame的列加上⼀个前缀，以便能
够跟其他数据进⾏合并。 get_dummies的prefix参数可以实现该
功能：

In [114]:
dummies = pd.get_dummies(df['key'], prefix='key')
dummies

,key_a,key_b,key_c
0,0,1,0
1,0,1,0
2,1,0,0
3,0,0,1
4,1,0,0
5,0,1,0


In [115]:
df

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,b,5


In [117]:
df_with_dummy = df[['data1']].join(dummies)
df_with_dummy

,data1,key_a,key_b,key_c
0,0,0,1,0
1,1,0,1,0
2,2,1,0,0
3,3,0,0,1
4,4,1,0,0
5,5,0,1,0
